In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd

import kraft

In [ ]:
maf = pd.read_csv("CCLE_mutations.csv", sep="\t")

maf

In [ ]:
maf = maf[
    [
        "Hugo_Symbol",
        "Variant_Classification",
        "Genome_Change",
        "Protein_Change",
        "Variant_annotation",
        "DepMap_ID",
    ]
]

maf = maf[maf["Variant_annotation"] == "damaging"]

maf["Cell Line"] = kraft.normalize_cell_line_names(maf["DepMap_ID"])

maf

In [ ]:
genes = np.unique(maf["Hugo_Symbol"].values)

dict_gene_i = kraft.make_dict_object_i(genes)

print(len(genes))

genes

In [ ]:
def format_(gene_str):

    gene, str_ = gene_str

    if not isinstance(str_, str):

        str_ = ""

    return np.asarray("{}_{}".format(gene, str_), dtype="object")

In [ ]:
variant_classifications = np.unique(
    np.apply_along_axis(
        format_, 1, maf[["Hugo_Symbol", "Variant_Classification"]].values,
    )
)

dict_variant_classification_i = kraft.make_dict_object_i(variant_classifications)


print(len(variant_classifications))

variant_classifications

In [ ]:
protein_changes = np.unique(
    np.apply_along_axis(format_, 1, maf[["Hugo_Symbol", "Protein_Change"]].values)
)

dict_protein_change_i = kraft.make_dict_object_i(protein_changes)

print(len(protein_changes))

protein_changes

In [ ]:
protein_changes = np.unique(
    np.apply_along_axis(format_, 1, maf[["Hugo_Symbol", "Protein_Change"]].values)
)

dict_protein_change_i = kraft.make_dict_object_i(protein_changes)

protein_changes

In [ ]:
cell_lines = np.unique(maf["Cell Line"].values)

dict_cell_line_i = kraft.make_dict_object_i(cell_lines)

cell_lines

In [ ]:
gene_x_cell_line = np.full((len(genes), len(cell_lines)), 0)

variant_classification_x_cell_line = np.full(
    (len(variant_classifications), len(cell_lines)), 0
)

protein_change_x_cell_line = np.full((len(protein_changes), len(cell_lines)), 0)

In [ ]:
for gene, variant_classification, protein_change, cell_line in maf[
    ["Hugo_Symbol", "Variant_Classification", "Protein_Change", "Cell Line"]
].values:

    gene_i = dict_gene_i[gene]

    cell_line_i = dict_cell_line_i[cell_line]

    gene_x_cell_line[gene_i, cell_line_i] += 1

    if not isinstance(variant_classification, str):

        variant_classification = ""

    variant_classification_i = dict_variant_classification_i[
        str(format_((gene, variant_classification)))
    ]

    variant_classification_x_cell_line[variant_classification_i, cell_line_i] += 1

    if not isinstance(protein_change, str):

        protein_change = ""

    protein_change_i = dict_protein_change_i[str(format_((gene, protein_change)))]

    protein_change_x_cell_line[protein_change_i, cell_line_i] += 1

In [ ]:
gene_x_cell_line = pd.DataFrame(gene_x_cell_line, index=genes, columns=cell_lines)

gene_x_cell_line.to_csv("mutation.gene_x_cell_line.tsv", sep="\t")

gene_x_cell_line

In [ ]:
gene_x_cell_line_01 = gene_x_cell_line.clip(upper=1)

gene_x_cell_line_01.to_csv("mutation.gene_x_cell_line.01.tsv", sep="\t")

gene_x_cell_line_01

In [ ]:
variant_classification_x_cell_line = pd.DataFrame(
    variant_classification_x_cell_line,
    index=variant_classifications,
    columns=cell_lines,
)

variant_classification_x_cell_line.to_csv(
    "gene_variant_classification_x_cell_line.tsv", sep="\t"
)

variant_classification_x_cell_line

In [ ]:
protein_change_x_cell_line = pd.DataFrame(
    protein_change_x_cell_line, index=protein_changes, columns=cell_lines
)

protein_change_x_cell_line.to_csv("gene_protein_change_x_cell_line.tsv", sep="\t")

protein_change_x_cell_line